In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [6]:
import pandas as pd
import folium
from folium import plugins
import requests, json
from datetime import datetime

colors = ['rgb(171,221,164)','rgb(254,224,139)','rgb(253,174,97)','rgb(244,109,67)','rgb(213,62,79)']
def get_color(f, volume, location):
    """get marker color based on traffic volume"""
    q = f[(f['location_name']==location)].Volume.quantile([0.25,0.5,0.75,0.9]).values
    if volume < q[0]:
        return colors[0]
    elif volume > q[0] and volume < q[1]:    
        return colors[1]
    elif volume > q[1] and volume < q[2]:    
        return colors[2]
    elif volume > q[2] and volume < q[3]:    
        return colors[3]
    else:
        return colors[4]

# Lane coordinates
lane_coord = {"2021 BLK KINNEY AVE (NW 300ft NW of Lamar)": {"None": [[-97.769321, 30.250531], [-97.770732, 30.248283]]}, "CAPITAL OF TEXAS HWY / LAKEWOOD DR": {"NB": [[-97.783021, 30.373691], [-97.787913, 30.369489]], "SB": [[-97.783289, 30.373951], [-97.788245, 30.369813]]}, "400 BLK AZIE MORTON RD (South of Barton Springs Rd)": {"SB": [[-97.765106, 30.264723], [-97.766051, 30.264064]], "NB": [[-97.765065, 30.264691], [-97.76601, 30.264034]]}, "BURNET RD / PALM WAY (IBM DRIVEWAY)": {"SB": [[-97.717408, 30.40287], [-97.718401, 30.401129]], "NB": [[-97.717271, 30.402814], [-97.718264, 30.401063]]}, "1000 BLK W CESAR CHAVEZ ST (H&B Trail Underpass)": {"EB": [[-97.761858, 30.269488], [-97.75866, 30.268068]], "WB": [[-97.761788, 30.269595], [-97.758603, 30.268185]]}, "LAMAR BLVD / SANDRA MURAIDA WAY (Lamar Bridge)": {"NB": [[-97.7570102, 30.2646134], [-97.7569485, 30.2647964], [-97.7568975, 30.2649331], [-97.75686, 30.2650814], [-97.7563369, 30.2663393], [-97.7562833, 30.2664528], [-97.7562055, 30.2666451], [-97.756117, 30.2668397], [-97.7555618, 30.2680327]], "SB": [[-97.7556771, 30.2680188], [-97.7563637, 30.2664899], [-97.7569056, 30.265174], [-97.7569243, 30.2651184], [-97.7569377, 30.2650651], [-97.7569672, 30.2649725], [-97.7571255, 30.2646389]]}, "100 BLK S CONGRESS AVE (Congress Bridge)": {"NB": [[-97.745429, 30.260732], [-97.746706, 30.257173]], "SB": [[-97.745665, 30.260834], [-97.746931, 30.257387]]}, "LAMAR BLVD / N LAMAR SB TO W 15TH RAMP": {"NB": [[-97.750308, 30.280748], [-97.750542, 30.278855]], "SB": [[-97.750682, 30.280951], [-97.750878, 30.278861]]}, "CONGRESS AVE / JOHANNA ST (Fulmore Middle School)": {"SB": [[-97.751431, 30.245143], [-97.752028, 30.24352]], "NB": [[-97.751262, 30.245105], [-97.751891, 30.243416]]}, "1612 BLK S LAMAR BLVD (Collier)": {"SB": [[-97.764003, 30.252651], [-97.76661, 30.250158]], "NB": [[-97.763778, 30.252465], [-97.766374, 30.249945]]}, "LAMAR BLVD / SHOAL CREEK BLVD": {"SB": [[-97.7487034, 30.2957997], [-97.7479631, 30.2946418], [-97.7472174, 30.2935162], [-97.747137, 30.2933356], [-97.7470994, 30.2931827], [-97.7470994, 30.2930577], [-97.7471048, 30.2929372], [-97.7470994, 30.2928677], [-97.7471155, 30.2927983], [-97.7471262, 30.2927334], [-97.7471638, 30.29265], [-97.7472442, 30.2925342], [-97.7473462, 30.2924462], [-97.7474213, 30.2923768], [-97.7474749, 30.2923397], [-97.7475607, 30.292298], [-97.7476466, 30.2922471], [-97.7477646, 30.29221], [-97.7486658, 30.2920201], [-97.7498835, 30.2917885]], "NB": [[-97.7498567, 30.2916727], [-97.7488375, 30.2918533], [-97.748124, 30.2920016], [-97.7478504, 30.2920618], [-97.7477056, 30.2920988], [-97.7475768, 30.2921452], [-97.7474481, 30.2922054], [-97.7473408, 30.2922795], [-97.7472496, 30.292349], [-97.7471799, 30.2924416], [-97.747094, 30.2925481], [-97.747035, 30.2926593], [-97.7469867, 30.2927797], [-97.7469546, 30.2929048], [-97.7469385, 30.293016], [-97.7469492, 30.2931318], [-97.7469492, 30.2932059], [-97.7469707, 30.2932846], [-97.7469975, 30.2933634], [-97.747035, 30.2934421], [-97.7472764, 30.2938451], [-97.7485693, 30.2958368]]}, "700 BLK E CESAR CHAVEZ ST": {"WB": [[-97.738738, 30.261854], [-97.736822, 30.261315]], "EB": [[-97.738778, 30.261727], [-97.736862, 30.261177]]}, "LAMAR BLVD / MANCHACA RD": {"NB": [[-97.780499, 30.244798], [-97.78322, 30.242842]], "SB": [[-97.780651, 30.244934], [-97.783367, 30.242983]]}, "CAPITAL OF TEXAS HWY / WALSH TARLTON LN": {"SB": [[-97.816003, 30.262221], [-97.810875, 30.255669]], "NB": [[-97.815402, 30.262499], [-97.810349, 30.256031]]}, "LAMAR BLVD / ZENNIA ST": {"SB": [[-97.729588, 30.321111], [-97.73158, 30.317937]], "NB": [[-97.729272, 30.321058], [-97.731325, 30.31777]]}, "BURNET RD / RUTLAND DR": {"NB": [[-97.72312, 30.386833], [-97.72461, 30.380611]], "SB": [[-97.723421, 30.386971], [-97.724816, 30.3809]]}, "CAPITAL OF TEXAS HWY / CEDAR ST": {"SB": [[-97.802593, 30.341246], [-97.804953, 30.337514]], "NB": [[-97.802292, 30.341172], [-97.804738, 30.337357]]}, "3201 BLK S LAMAR BLVD (BROKEN SPOKE)": {"None": [[-97.78467, 30.241911], [-97.789262, 30.238682]]}}

In [94]:
import torch
from torch import nn, optim
import numpy as np
from sklearn.preprocessing import MinMaxScaler


dtc_data = pd.read_csv(r'Radar_Traffic_Counts.csv')

dtc_data['location_name'] = dtc_data \
    .location_name.apply(lambda x: x.strip())

dtc_data['Date'] = pd.to_datetime(dtc_data[['Year', 'Month', 'Day']])  # ,'Hour']])

# traffic for each crossroad

locations = list(dtc_data.location_name.unique())
hourly_locations = dict()
for i in range(len(locations)):
    hourly_locations[i] = dtc_data[dtc_data['location_name'] == locations[i]] \
        .groupby(['Hour']) \
        .agg({'Volume': 'mean'}) \
        .reset_index()['Volume']
    #hourly_locations[i] = hourly_locations[i].diff().fillna(hourly_locations[i][0]).astype(np.int64)

def test_train_scaling(dataset):
    test_data_size = 8
    train_data = dataset[:-test_data_size]
    test_data = dataset[-test_data_size:]
    # normalizing data
    scaler = MinMaxScaler()
    scaler = scaler.fit(np.expand_dims(train_data, axis=1))
    train_data = scaler.transform(np.expand_dims(train_data, axis=1))
    test_data = scaler.transform(np.expand_dims(test_data, axis=1))
    return train_data, test_data, scaler


# normalising sequence data
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)


seq_length = 5
def sampling_data(train_data, test_data):
    X_train, y_train = create_sequences(train_data, seq_length)
    X_test, y_test = create_sequences(test_data, seq_length)    
    X_train = torch.from_numpy(X_train).float()
    y_train = torch.from_numpy(y_train).float()

    X_test = torch.from_numpy(X_test).float()
    y_test = torch.from_numpy(y_test).float()
    return X_train, y_train, X_test, y_test


class TrafficPredictor(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers):
        super(TrafficPredictor,self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            dropout=0.1
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)

    def forward(self, sequences):
        lstm_out, self.hidden = self.lstm(
            sequences.view(len(sequences), self.seq_len, -1),
            self.hidden
        )
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred
    def reset_hidden_state(self):
        self.hidden = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
        )

def train_model(model,train_data,train_labels, test_data=None,test_labels=None):
    loss_fn = torch.nn.MSELoss(reduction='sum')

    optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
    num_epochs = 200

    train_hist = np.zeros(num_epochs)
    test_hist = np.zeros(num_epochs)

    for t in range(num_epochs):
        model.reset_hidden_state()

        y_pred = model(X_train)

        loss = loss_fn(y_pred.float(), y_train)

        if test_data is not None:
            with torch.no_grad():
                y_test_pred = model(X_test)
                test_loss = loss_fn(y_test_pred.float(), y_test)
            test_hist[t] = test_loss.item()

            if t % 10 == 0:
                print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
        elif t % 10 == 0:
            print(f'Epoch {t} train loss: {loss.item()}')

        train_hist[t] = loss.item()

        optimiser.zero_grad()

        loss.backward()

        optimiser.step()

    return model.eval(), train_hist, test_hist

true_cases = dict()
predicted_cases = dict()
for z in range(len(hourly_locations)):
    train_data, test_data, scaler = test_train_scaling(hourly_locations[z])
    X_train, y_train, X_test, y_test = sampling_data(train_data, test_data )
    model = TrafficPredictor(n_features=1, n_hidden=512, seq_len=seq_length, n_layers=2)
    model, train_hist, test_hist = train_model(model, X_train, y_train, X_test, y_test)
    
    with torch.no_grad():
      test_seq = X_test[:1]
      preds = []
      for _ in range(len(X_test)):
        y_test_pred = model(test_seq)
        pred = torch.flatten(y_test_pred).item()
        preds.append(pred)
        new_seq = test_seq.numpy().flatten()
        new_seq = np.append(new_seq, [pred])
        new_seq = new_seq[1:]
        test_seq = torch.as_tensor(new_seq).view(1, seq_length, 1).float()

    true_cases[z] = scaler.inverse_transform(np.expand_dims(y_test.flatten().numpy(), axis=0)).flatten()
    predicted_cases[z] = scaler.inverse_transform(np.expand_dims(preds, axis=0)).flatten()

Epoch 0 train loss: 3.938929796218872 test loss: 0.34007781744003296
Epoch 10 train loss: 0.9956720471382141 test loss: 0.05270218476653099
Epoch 20 train loss: 0.6228342056274414 test loss: 0.4907374680042267
Epoch 30 train loss: 0.39655810594558716 test loss: 1.7341341972351074
Epoch 40 train loss: 0.33940407633781433 test loss: 0.6388479471206665
Epoch 50 train loss: 0.315873920917511 test loss: 0.2180759459733963
Epoch 60 train loss: 0.31614920496940613 test loss: 0.24587830901145935
Epoch 70 train loss: 0.2981928884983063 test loss: 0.33573734760284424
Epoch 80 train loss: 0.291347473859787 test loss: 0.29407772421836853
Epoch 90 train loss: 0.2851715385913849 test loss: 0.17899297177791595
Epoch 100 train loss: 0.2727784514427185 test loss: 0.1545383185148239
Epoch 110 train loss: 0.22163675725460052 test loss: 0.172464057803154
Epoch 120 train loss: 0.12011346220970154 test loss: 1.2717876434326172
Epoch 130 train loss: 0.501128613948822 test loss: 1.5445560216903687
Epoch 140 t

Epoch 160 train loss: 0.06959758698940277 test loss: 0.7949401140213013
Epoch 170 train loss: 0.04994823783636093 test loss: 0.8820960521697998
Epoch 180 train loss: 0.05773623660206795 test loss: 0.8977631330490112
Epoch 190 train loss: 0.01705748774111271 test loss: 0.8128551244735718
Epoch 0 train loss: 5.017011642456055 test loss: 0.6656626462936401
Epoch 10 train loss: 1.8591315746307373 test loss: 0.10687482357025146
Epoch 20 train loss: 1.1017507314682007 test loss: 0.31295907497406006
Epoch 30 train loss: 0.7911276817321777 test loss: 0.03911787271499634
Epoch 40 train loss: 0.5828500390052795 test loss: 0.19455820322036743
Epoch 50 train loss: 0.38368239998817444 test loss: 1.7786320447921753
Epoch 60 train loss: 0.3096255958080292 test loss: 0.8304280042648315
Epoch 70 train loss: 0.23363614082336426 test loss: 0.5777084231376648
Epoch 80 train loss: 0.1367526650428772 test loss: 0.17263364791870117
Epoch 90 train loss: 0.10794132202863693 test loss: 0.05007602646946907
Epoch

Epoch 110 train loss: 0.25605353713035583 test loss: 0.3405758738517761
Epoch 120 train loss: 0.19608856737613678 test loss: 0.18656188249588013
Epoch 130 train loss: 0.0850101038813591 test loss: 0.038593634963035583
Epoch 140 train loss: 0.06368011236190796 test loss: 0.02505955658853054
Epoch 150 train loss: 0.0592067688703537 test loss: 0.016418032348155975
Epoch 160 train loss: 0.0368637777864933 test loss: 0.004339359235018492
Epoch 170 train loss: 0.045812785625457764 test loss: 0.05806754529476166
Epoch 180 train loss: 0.03430023044347763 test loss: 0.08461950719356537
Epoch 190 train loss: 0.014345522969961166 test loss: 0.2769942581653595
Epoch 0 train loss: 4.895296573638916 test loss: 0.4115079343318939
Epoch 10 train loss: 1.7378695011138916 test loss: 0.02442256361246109
Epoch 20 train loss: 0.8829541206359863 test loss: 0.31114375591278076
Epoch 30 train loss: 0.7206944227218628 test loss: 0.4062016010284424
Epoch 40 train loss: 0.5196554064750671 test loss: 1.1839121580

Epoch 70 train loss: 0.07557439059019089 test loss: 0.6940281391143799
Epoch 80 train loss: 0.08619128912687302 test loss: 0.6979939937591553
Epoch 90 train loss: 0.057334352284669876 test loss: 0.8639800548553467
Epoch 100 train loss: 0.031651489436626434 test loss: 0.7021785974502563
Epoch 110 train loss: 0.03382546827197075 test loss: 0.5179836750030518
Epoch 120 train loss: 0.025648271664977074 test loss: 0.4831162691116333
Epoch 130 train loss: 0.028470568358898163 test loss: 0.45986440777778625
Epoch 140 train loss: 0.021741420030593872 test loss: 0.4328656792640686
Epoch 150 train loss: 0.024021895602345467 test loss: 0.3488066792488098
Epoch 160 train loss: 0.016381051391363144 test loss: 0.27904486656188965
Epoch 170 train loss: 0.018708735704421997 test loss: 0.19505241513252258
Epoch 180 train loss: 0.017434947192668915 test loss: 0.16552725434303284
Epoch 190 train loss: 0.016212623566389084 test loss: 0.1492345631122589


In [95]:
np_dict = dict()
predicted = dict()
for a in range(len(hourly_locations)):
    hourly_locations[a] = np.array(hourly_locations[a])
    np_dict[a] = np.append(hourly_locations[a][:-2] ,true_cases[a])
    

In [96]:
#np.concatenate(np_dict.values(), axis = 0)
#volume = np.concatenate([np_dict], axis = 0)
#np.array([np_dict.values()])
volume = np.concatenate(np.vstack(np_dict.values()), axis=0)

/Users/apple/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


### Visualize time series traffic data on a map

Load and cleanup data from csv file.

In [97]:
df = pd.read_csv("Radar_Traffic_Counts.csv")
# cleanup leading space in names
df['location_name']=df.location_name.apply(lambda x: x.strip()) 
df.sample()

,location_name,location_latitude,location_longitude,Year,Month,Day,Day of Week,Hour,Minute,Time Bin,Direction,Volume
525030,1000 BLK W CESAR CHAVEZ ST (H&B Trail Underpass),30.268652,-97.759929,2018,10,9,2,8,45,08:45,EB,241


### Aggregate data

Aggregate data to get mean traffic in each direction, by the hour, at all sensor locations 

In [162]:
hourly_vol = df.groupby(['location_name','location_latitude','location_longitude','Hour']) \
    .agg({'Volume':'mean'}) \
    .reset_index()
ran_dir = df \
    .groupby(['location_name','Direction']) \
    .agg({'Volume':'mean'}) \
    .drop(['Volume'], axis = 1) \
    .reset_index()
ran_dir = ran_dir[ran_dir.location_name.isin(locations)].drop_duplicates(subset='location_name').reset_index().drop(['index'], axis = 1)
hourly_vol = pd.merge(hourly_vol, ran_dir, how = 'left', on = 'location_name', copy = False)

#hourly_vol['Direction'] = None
hourly_vol['Volume'] = volume
cols = ['location_name',
 'location_latitude',
 'location_longitude',
 'Direction',       
 'Hour',
 'Volume'
 ]
hourly_vol= hourly_vol[cols]


### Timestamped GeoJSON


In [161]:
# Date slider - We are visualizing volume at each location by hour of day, using several years of data. Date can be ignored.
dt = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)

# construct geo json
lanes = [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": lane_coord[row[0]][row[3]],
      },
      "properties": {
        'style':{
          'color': get_color(hourly_vol,row[5],row[0]),
          'weight': 3
        },
        "times": [dt.replace(hour=row[4], second=i).strftime("%Y-%m-%dT%T") for i in range(len(lane_coord[row[0]][row[3]]))]
      }
    }
    for idx, row in enumerate(hourly_vol.values)
  ]

data = {
  "type": "FeatureCollection",
  "features": lanes
}

m = folium.Map([30.264428, -97.751054], 
               tiles='CartoDB positron', 
               zoom_start=16,
               control_scale=True,
               attr='Radar traffic sensor data<br>from <a href="https://data.austintexas.gov/">City of Austin</a>\'s public dataset'
              )
m.add_child(plugins.TimestampedGeoJson(data
                                       ,period='PT1H'
                                       ,date_options='HH:00'
                                       ,transition_time=500
                                       ,add_last_point=False)
)
m